In [1]:
# EDA를 위한 분석 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#검증 데이터 분리를 위한 함수
from sklearn.model_selection import train_test_split
#학습을 위한 모델 클래스
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
#평가를 위한 함수(정확도)
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv("./data/salary/train.csv", index_col="no")
test = pd.read_csv("./data/salary/test.csv" , index_col="no")
display("Train")
display(train.head())
display("Test")
display(test.head())

'Train'

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
no,,,,,,,,,,,,,,,
1,25,Private,219199,11th,7,Divorced,Machine-op-inspct,Not-in-family,White,Male,0,0,40,United-States,0
2,39,Private,52978,Some-college,10,Divorced,Other-service,Not-in-family,White,Female,0,1721,55,United-States,0
3,35,Private,196899,Bachelors,13,Never-married,Handlers-cleaners,Not-in-family,Asian-Pac-Islander,Female,0,0,50,Haiti,0
4,64,Private,135527,Assoc-voc,11,Divorced,Tech-support,Not-in-family,White,Female,0,0,40,United-States,0
5,24,Private,60783,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,70,United-States,1


'Test'

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
no,,,,,,,,,,,,,,
29306,18,?,245274,Some-college,10,Never-married,?,Own-child,White,Male,0,0,16,United-States
29307,29,Private,83003,HS-grad,9,Married-civ-spouse,Other-service,Wife,White,Female,0,0,40,United-States
29308,45,Private,35136,Bachelors,13,Married-civ-spouse,Tech-support,Husband,Black,Male,0,0,40,United-States
29309,42,Self-emp-not-inc,64631,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
29310,41,Private,195821,Doctorate,16,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,1902,40,United-States


In [3]:
# 전체 데이터 수 파악
train.shape, test.shape

((29305, 15), (19537, 14))

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29305 entries, 1 to 29305
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             29305 non-null  int64 
 1   workclass       29305 non-null  object
 2   fnlwgt          29305 non-null  int64 
 3   education       29305 non-null  object
 4   education-num   29305 non-null  int64 
 5   marital-status  29305 non-null  object
 6   occupation      29305 non-null  object
 7   relationship    29305 non-null  object
 8   race            29305 non-null  object
 9   sex             29305 non-null  object
 10  capital-gain    29305 non-null  int64 
 11  capital-loss    29305 non-null  int64 
 12  hours-per-week  29305 non-null  int64 
 13  native-country  29305 non-null  object
 14  income          29305 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 3.6+ MB


In [5]:
# 범주형 데이터
# workclass, education, marital-status, occupation, relationship, race, sex, native-country
# 수치형 데이터
# age, fnlwgt, education-num, capital-gain, capital-loss, hours-per-week

In [6]:
display(train.corr()) # 상관계수 확인

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income
age,1.000000,-0.075753,0.035084,0.078498,0.054413,0.073100,0.238460
fnlwgt,-0.075753,1.000000,-0.030600,-0.005051,-0.001513,-0.010154,-0.002994
education-num,0.035084,-0.030600,1.000000,0.127651,0.083925,0.147569,0.331798
capital-gain,0.078498,-0.005051,0.127651,1.000000,-0.031401,0.088609,0.221387
capital-loss,0.054413,-0.001513,0.083925,-0.031401,1.000000,0.055271,0.135645
hours-per-week,0.073100,-0.010154,0.147569,0.088609,0.055271,1.000000,0.231045
income,0.238460,-0.002994,0.331798,0.221387,0.135645,0.231045,1.000000


In [7]:
# 결측치 확인(결측치 null이 아닌 " ?"로 처리되어있음) 
# 숫자형에는 결측치 없음
# workclass(1663), occupation(1668), native-country(495) 결측치 있음
# workclass - private 수가 너무 많아서 의미 없을거 같음(결측치private로 처리)
# occupation - 결측치 처리 하지 않고 ? 그대로 사용할것임 -> 3~4단계로 분류
# native-country - United-States수가 너무 많아서 의미 없을거 같음(결측치United-States로 처리)
train[train['native-country']==" ?"].count()


age               495
workclass         495
fnlwgt            495
education         495
education-num     495
marital-status    495
occupation        495
relationship      495
race              495
sex               495
capital-gain      495
capital-loss      495
hours-per-week    495
native-country    495
income            495
dtype: int64

In [8]:
train['native-country']=train['native-country'].str.replace("?","United-States")
train['workclass']=train['workclass'].str.replace("?","private")

C:\Users\pipi1\AppData\Local\Temp\ipykernel_11668\1488058291.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['native-country']=train['native-country'].str.replace("?","United-States")
C:\Users\pipi1\AppData\Local\Temp\ipykernel_11668\1488058291.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['workclass']=train['workclass'].str.replace("?","private")


In [9]:
test['native-country']=test['native-country'].str.replace("?","United-States")
test['workclass']=test['workclass'].str.replace("?","private")

C:\Users\pipi1\AppData\Local\Temp\ipykernel_11668\1070983275.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test['native-country']=test['native-country'].str.replace("?","United-States")
C:\Users\pipi1\AppData\Local\Temp\ipykernel_11668\1070983275.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test['workclass']=test['workclass'].str.replace("?","private")


In [10]:
train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
no,,,,,,,,,,,,,,,
1,25,Private,219199,11th,7,Divorced,Machine-op-inspct,Not-in-family,White,Male,0,0,40,United-States,0
2,39,Private,52978,Some-college,10,Divorced,Other-service,Not-in-family,White,Female,0,1721,55,United-States,0
3,35,Private,196899,Bachelors,13,Never-married,Handlers-cleaners,Not-in-family,Asian-Pac-Islander,Female,0,0,50,Haiti,0
4,64,Private,135527,Assoc-voc,11,Divorced,Tech-support,Not-in-family,White,Female,0,0,40,United-States,0
5,24,Private,60783,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,70,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29301,20,Private,100605,HS-grad,9,Never-married,Sales,Own-child,Other,Male,0,0,40,Puerto-Rico,0
29302,21,Private,372636,HS-grad,9,Never-married,Sales,Own-child,Black,Male,0,0,40,United-States,0
29303,18,Self-emp-not-inc,258474,10th,6,Never-married,Farming-fishing,Own-child,White,Male,0,0,40,United-States,0


In [11]:
# 인코딩
# workclass - 의미X
# education - education-num이랑 비율같아서 인코딩 필요 없음
# race - white 비율 너무 높아서 의미X
# native-country - United-States 비율 너무 높아서 의미X
# 원핫인코딩 marital-status, occupation, relationship, sex
# 라벨인코딩 marital-status,occupation(income 과 비교해서 라벨링 해보기)

In [12]:
train['native-country'].value_counts()

 United-States                 26796
 Mexico                          579
 Philippines                     178
 Germany                         124
 Puerto-Rico                     117
 Canada                          111
 El-Salvador                      96
 Cuba                             94
 India                            90
 England                          75
 South                            74
 Jamaica                          68
 China                            68
 Dominican-Republic               61
 Italy                            60
 Japan                            54
 Columbia                         53
 Poland                           53
 Guatemala                        51
 Haiti                            51
 Vietnam                          50
 Portugal                         43
 Taiwan                           39
 Iran                             38
 Nicaragua                        28
 Peru                             25
 Greece                           25
 

In [13]:
list=np.arange(1,16)
list=list.tolist()
list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [ ]:
train[['occupation','income']].groupby(['occupation']).mean().sort_values(by='income')

In [14]:
# 라벨인코딩 marital-status,occupation(income 과 비교해서 라벨링 해보기)
# occupation
a=(train[['occupation','income']].groupby(['occupation']).mean().sort_values(by="income").reset_index())
a['occupation'].to_dict()

{0: ' Priv-house-serv',
 1: ' Other-service',
 2: ' Handlers-cleaners',
 3: ' ?',
 4: ' Farming-fishing',
 5: ' Machine-op-inspct',
 6: ' Adm-clerical',
 7: ' Transport-moving',
 8: ' Craft-repair',
 9: ' Sales',
 10: ' Tech-support',
 11: ' Armed-Forces',
 12: ' Protective-serv',
 13: ' Prof-specialty',
 14: ' Exec-managerial'}

In [15]:
method_dict = {
    ' Priv-house-serv':0,
    ' Other-service':0,
    ' Handlers-cleaners':0,
    ' ?':1,
    ' Farming-fishing':1,
    ' Machine-op-inspct':1,
    ' Adm-clerical':1,
    ' Transport-moving':2,
    ' Craft-repair':2,
    ' Sales':2,
    ' Tech-support':2,
    ' Armed-Forces':3,
    ' Protective-serv':3,
    ' Prof-specialty':4,
    ' Exec-managerial':4
    }
train['occupation']=train['occupation'].map(method_dict)
test['occupation']=test['occupation'].map(method_dict)

In [16]:
train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
no,,,,,,,,,,,,,,,
1,25,Private,219199,11th,7,Divorced,5,Not-in-family,White,Male,0,0,40,United-States,0
2,39,Private,52978,Some-college,10,Divorced,1,Not-in-family,White,Female,0,1721,55,United-States,0
3,35,Private,196899,Bachelors,13,Never-married,2,Not-in-family,Asian-Pac-Islander,Female,0,0,50,Haiti,0
4,64,Private,135527,Assoc-voc,11,Divorced,10,Not-in-family,White,Female,0,0,40,United-States,0
5,24,Private,60783,Some-college,10,Married-civ-spouse,7,Husband,White,Male,0,0,70,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29301,20,Private,100605,HS-grad,9,Never-married,9,Own-child,Other,Male,0,0,40,Puerto-Rico,0
29302,21,Private,372636,HS-grad,9,Never-married,9,Own-child,Black,Male,0,0,40,United-States,0
29303,18,Self-emp-not-inc,258474,10th,6,Never-married,4,Own-child,White,Male,0,0,40,United-States,0


In [17]:
# marital-status
a=(train[['marital-status','income']].groupby(['marital-status']).mean().sort_values(by="income").reset_index())
a

,marital-status,income
0,Never-married,0.046831
1,Separated,0.067834
2,Widowed,0.083864
3,Married-spouse-absent,0.093909
4,Divorced,0.104467
5,Married-AF-spouse,0.380952
6,Married-civ-spouse,0.448954


In [18]:
train['marital-status']

no
1                   Divorced
2                   Divorced
3              Never-married
4                   Divorced
5         Married-civ-spouse
                ...         
29301          Never-married
29302          Never-married
29303          Never-married
29304          Never-married
29305                Widowed
Name: marital-status, Length: 29305, dtype: object

In [19]:
method_dict = {
    ' Never-married':0,
    ' Separated':1,
    ' Widowed':2,
    ' Married-spouse-absent':3,
    ' Divorced':4,
    ' Married-AF-spouse':5,
    ' Married-civ-spouse':6
 }
train['marital-status']=train['marital-status'].map(method_dict)
test['marital-status']=test['marital-status'].map(method_dict)


In [20]:
train[['marital-status','income']].groupby(['marital-status']).mean().sort_values(by="income")

,income
marital-status,
0,0.046831
1,0.067834
2,0.083864
3,0.093909
4,0.104467
5,0.380952
6,0.448954


In [ ]:
# 나이 구간으로 나눔
# 이상 미만
age_all_=[]
age_income_=[]
age_=[]
for i in range(15,101,5):
    a=train[train['age']<i+5].count()['age']-train[train['age']<=i].count()['age']
    age_all_.append(a)
    b=train[train['age']<i+5][train['income']==1].count()['age']-train[train['age']<=i][train['income']==1].count()['age']
    age_income_.append(b)
    age_.append(str(i)+'~'+str(i+5))

In [ ]:
data3=pd.DataFrame([age_all_,age_income_], columns=age_).T
data3

In [ ]:
data3[2]=data3[1]/data3[0]
data3

In [ ]:
#구간화 작업
bins=[15,20,25,35,40,55,60,90,100]  #(이상,미만)
cat=[1,2,3,4,5,6,7,8]
age_cut = pd.cut(train['age'],bins=bins,labels=cat)
age_cut

In [ ]:
train['age_cut'] = age_cut

In [ ]:
# test 데이터 적용
age_cut_test= pd.cut(test['age'],bins=bins,labels=cat)
test['age_cut'] = age_cut_test

In [21]:
# 정답 분리
X = train.iloc[:,:-1]
y = train.iloc[:,-1] 

In [22]:
#원핫 인코딩  marital-status, occupation, relationship, sex
feature_names=['marital-status', 'occupation', 'relationship', 'sex']
train_one_hot = pd.get_dummies(X[feature_names])
train_one_hot

,marital-status,occupation,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,sex_ Female,sex_ Male
no,,,,,,,,,,
1,4,5,0,1,0,0,0,0,0,1
2,4,1,0,1,0,0,0,0,1,0
3,0,2,0,1,0,0,0,0,1,0
4,4,10,0,1,0,0,0,0,1,0
5,6,7,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
29301,0,9,0,0,0,1,0,0,0,1
29302,0,9,0,0,0,1,0,0,0,1
29303,0,4,0,0,0,1,0,0,0,1


In [23]:
# test처리
test_one_hot = pd.get_dummies(test[feature_names])
test_one_hot

,marital-status,occupation,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,sex_ Female,sex_ Male
no,,,,,,,,,,
29306,0,3,0,0,0,1,0,0,0,1
29307,6,1,0,0,0,0,0,1,1,0
29308,6,10,1,0,0,0,0,0,0,1
29309,6,14,1,0,0,0,0,0,0,1
29310,6,14,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
48838,6,8,1,0,0,0,0,0,0,1
48839,0,14,0,1,0,0,0,0,1,0
48840,6,13,1,0,0,0,0,0,0,1


In [24]:
# 원핫인코딩한거 붙이기
train = pd.concat([X,train_one_hot], axis=1)
test = pd.concat([test,test_one_hot], axis=1)

In [25]:
train.drop(feature_names,axis=1,inplace=True)
test.drop(feature_names,axis=1,inplace=True)

In [26]:
train.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'race',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'relationship_ Husband', 'relationship_ Not-in-family',
       'relationship_ Other-relative', 'relationship_ Own-child',
       'relationship_ Unmarried', 'relationship_ Wife', 'sex_ Female',
       'sex_ Male'],
      dtype='object')

In [27]:
salary_model = KNeighborsClassifier()

In [34]:
X_train= train[['education-num','capital-loss', 'hours-per-week','capital-gain',
       'relationship_ Husband', 'relationship_ Wife', 'sex_ Female',
       'sex_ Male']]
y_train= y
X_train1,X_val,y_train1,y_val=train_test_split(X_train,y_train,test_size=0.2, random_state=7)                                                                                                                       
salary_model.fit(X_train1,y_train1)
pre= salary_model.predict(X_val)
pre
score = accuracy_score(y_val,pre)
print("정확도 : ",score)

정확도 :  0.8246033100153557


In [29]:
# 캐글 업로드
X_test= test[['education-num','capital-loss', 'hours-per-week',
       'relationship_ Husband', 'relationship_ Not-in-family',
       'relationship_ Other-relative', 'relationship_ Own-child',
       'relationship_ Unmarried', 'relationship_ Wife', 'sex_ Female',
       'sex_ Male']]

In [30]:
test_pre=salary_model.predict(X_test)
test_pre

array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [31]:
submission = pd.read_csv("./data/salary/sample_submission.csv")
submission

,no,income
0,29306,0
1,29307,0
2,29308,0
3,29309,0
4,29310,0
...,...,...
19532,48838,1
19533,48839,0
19534,48840,1
19535,48841,0


In [32]:
submission['income'] = test_pre
submission

,no,income
0,29306,0
1,29307,0
2,29308,1
3,29309,1
4,29310,1
...,...,...
19532,48838,1
19533,48839,0
19534,48840,1
19535,48841,0


In [33]:
# csv파일로 저장 index=False -> 이걸 안하면 인덱스도 컬럼으로 저장된다
submission.to_csv("./data/salary/myPrediction.csv", index=False)